In [1]:
import requests
import json
import pandas as pd
import xmltodict
from lxml.html import fromstring
import numpy as np
from datetime import datetime

In [2]:
start_time = datetime.now()

# Read the excel file

In [3]:
df = pd.read_excel('Disease test set.xlsx')
Disease = df[["Disease from model"]].to_numpy() 
Disease

array([['hypertension']], dtype=object)

# Declare the different Sources

In [4]:
source = n = np.array(['SNOMEDCT_US','MSH','OMIM','MDR'])

# Save the API Key and get the tgt

In [5]:
api_key = 'b97fa897-104d-4139-a516-47d8c31dbfee'
uri="https://utslogin.nlm.nih.gov"

In [6]:
h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
params = {'apikey': api_key}
auth_endpoint = "/cas/v1/api-key"
r = requests.post(uri+auth_endpoint,data=params,headers=h)
response = fromstring(r.text)
tgt = response.xpath('//form/@action')[0]

# Get Disease Codes from UMLS

In [7]:
def get_UMLS_codes(tgt,Disease):
    synonyms_list = np.array([])
    ui_list = np.array([])
    source_list = np.array([])
    disease_list = np.array([])
    
    for source_name in source:
        for disease_name in Disease:
              
            service="http://umlsks.nlm.nih.gov"
            params = {'service': service}
            h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
            r = requests.post(tgt,data=params,headers=h)
            st = r.text
            
            base_url = 'https://uts-ws.nlm.nih.gov/rest/search/current?string='+str(disease_name)+'&pageNumber=1&sabs='+source_name+'&returnIdType=sourceUi&sinputType=atom&includeObsolete=false&ticket='+st

            r = requests.get(base_url)
            data = r.json()
            jtopy=json.dumps(data)
            dict_json=json.loads(jtopy)
            n = len(dict_json['result']['results'])
            i = 0


            while (i < n):
                Synonyms  = str(dict_json['result']['results'][i]['name'])
                ui = str(dict_json['result']['results'][i]['ui'])
                synonyms_list = np.append(synonyms_list,Synonyms)
                ui_list = np.append(ui_list,str(ui))
                source_list = np.append(source_list,source_name)
                disease_list = np.append(disease_list,disease_name)
                i+=1


            
    Final_Array = np.array([source_list,disease_list,ui_list,synonyms_list])
    return Final_Array
    
            

# Get Parent Information

In [8]:
def get_UMLS_Parents(tgt,source,child_ui):
    
    service="http://umlsks.nlm.nih.gov"
    params = {'service': service}
    h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
    r = requests.post(tgt,data=params,headers=h)
    st = r.text
    base_url = 'https://uts-ws.nlm.nih.gov/rest/content/current/source/'+source+'/'+child_ui+'/parents?ticket='+st
    r = requests.get(base_url)
    data = r.json()
    jtopy=json.dumps(data)
    dict_json=json.loads(jtopy)
    n = len(dict_json['result'])
    i = 0 
    
    parents_list = np.array([])
    ui_list = np.array([])
    child_list= np.array([])
    source_list= np.array([])

    while (i < n ):
        parents = dict_json['result'][i]['name']
        ui = dict_json['result'][i]['ui']
        
        source_list = np.append(source_list,source)
        parents_list = np.append(parents_list,parents)
        ui_list = np.append(ui_list,ui)
        child_list = np.append(child_list,child_ui)
        i+=1
    Final_Array = np.array([source_list,child_list,ui_list,parents_list])
    
    return Final_Array
    

# Iterate through child's list

In [9]:
def itertate_child(tgt,child):
    i = 0
    temp_parent = np.array([])
    parent = np.array([])

    while ( i < len(child[0])):

        try:
            temp_parent = get_UMLS_Parents(tgt,child[0][i],child[2][i])
            temp_parent = temp_parent.reshape((4, -1))
            parent = parent.reshape((4, -1))
            parent= np.concatenate((parent,temp_parent),axis=1) 

        except:
            temp_parent=np.array([child[0][i],child[2][i],'None','None'])
            temp_parent = temp_parent.reshape((4,-1))
            parent = parent.reshape((4,-1))
            parent= np.concatenate((parent,temp_parent),axis=1) 

        i+=1
    return parent


In [87]:
synonyms = get_UMLS_codes(tgt,Disease)
source_df = pd.DataFrame(list(zip(synonyms[0],synonyms[1],synonyms[2],synonyms[3])))
source_df =source_df.rename(columns={ source_df.columns[0]: "Source" ,source_df.columns[1]: "Disease" , source_df.columns[2]: "UID" , source_df.columns[3]: "Synonyms"})
source_df = source_df.drop_duplicates(subset=['Source','Disease','UID','Synonyms'], keep='first')
source_df.head()

,Source,Disease,UID,Synonyms
0,SNOMEDCT_US,hypertension,34742003,Portal hypertension
1,SNOMEDCT_US,hypertension,70995007,Pulmonary hypertension
2,SNOMEDCT_US,hypertension,59621000,Essential hypertension
3,SNOMEDCT_US,hypertension,271719001,Raised intracranial pressure
4,SNOMEDCT_US,hypertension,123799005,Renovascular hypertension


In [60]:
#Total Number of rows from Synonyms Table
len(source_df)

100

# Get Parents 1 Information

In [61]:
p1 = itertate_child(tgt,synonyms)
p1_df = pd.DataFrame(list(zip(p1[0],p1[1],p1[2],p1[3])))
p1_df=p1_df.rename(columns={ p1_df.columns[0]: "Source" ,p1_df.columns[1]: "Child_UI" , p1_df.columns[2]: "Parent1_UI" , p1_df.columns[3]: "Parent1"})
p1_df.head()

,Source,Child_UI,Parent1_UI,Parent1
0,SNOMEDCT_US,34742003,234072000,Venous hypertension
1,SNOMEDCT_US,34742003,128056006,Disorder of portal venous system
2,SNOMEDCT_US,70995007,39785005,Disorder of pulmonary circulation
3,SNOMEDCT_US,59621000,38341003,Hypertensive disorder
4,SNOMEDCT_US,271719001,299732003,Finding of intracranial pressure


In [62]:
print(len(p1_df))
p1_df = p1_df.drop_duplicates(subset=['Source','Child_UI','Parent1_UI','Parent1'], keep='first')
print(len(p1_df))

123
123


In [88]:
source_df = pd.merge(source_df, p1_df, left_on=['Source','UID'], right_on=['Source','Child_UI'],how='left')
source_df = source_df.drop(['Child_UI'], axis = 1)
source_df.head()

,Source,Disease,UID,Synonyms,Parent1_UI,Parent1
0,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension
1,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system
2,SNOMEDCT_US,hypertension,70995007,Pulmonary hypertension,39785005,Disorder of pulmonary circulation
3,SNOMEDCT_US,hypertension,59621000,Essential hypertension,38341003,Hypertensive disorder
4,SNOMEDCT_US,hypertension,271719001,Raised intracranial pressure,299732003,Finding of intracranial pressure


# Get Parents 2 Information

In [64]:
p2 = itertate_child(tgt,p1)
p2_df = pd.DataFrame(list(zip(p2[0],p2[1],p2[2],p2[3])))
p2_df=p2_df.rename(columns={ p2_df.columns[0]: "Source" ,p2_df.columns[1]: "Child_UI" , p2_df.columns[2]: "Parent2_UI" , p2_df.columns[3]: "Parent2"})
p2_df.head()

,Source,Child_UI,Parent2_UI,Parent2
0,SNOMEDCT_US,234072000,90507008,Disorder of vein
1,SNOMEDCT_US,128056006,128052008,Vascular disease of abdomen
2,SNOMEDCT_US,128056006,90507008,Disorder of vein
3,SNOMEDCT_US,39785005,373434004,Disorder of blood vessels of thorax
4,SNOMEDCT_US,38341003,366157005,Cardiovascular measurement - finding


In [65]:
print(len(p2_df))
p2_df = p2_df.drop_duplicates(subset=['Source','Child_UI','Parent2_UI','Parent2'], keep='first')
print(len(p2_df))

150
85


In [66]:
dedup_p2 = np.stack([p2_df['Source'], p2_df['Child_UI'], p2_df['Parent2_UI'], p2_df['Parent2']])

In [89]:
source_df = pd.merge(source_df, p2_df, left_on=['Source','Parent1_UI'], right_on=['Source','Child_UI'],how='left')
source_df = source_df.drop(['Child_UI'], axis = 1)

source_df.head()

,Source,Disease,UID,Synonyms,Parent1_UI,Parent1,Parent2_UI,Parent2
0,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein
1,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,128052008,Vascular disease of abdomen
2,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,90507008,Disorder of vein
3,SNOMEDCT_US,hypertension,70995007,Pulmonary hypertension,39785005,Disorder of pulmonary circulation,373434004,Disorder of blood vessels of thorax
4,SNOMEDCT_US,hypertension,59621000,Essential hypertension,38341003,Hypertensive disorder,366157005,Cardiovascular measurement - finding


# Get Parents 3 Information

In [68]:
p3 = itertate_child(tgt,dedup_p2)
p3_df = pd.DataFrame(list(zip(p3[0],p3[1],p3[2],p3[3])))
p3_df=p3_df.rename(columns={ p3_df.columns[0]: "Source" ,p3_df.columns[1]: "Child_UI" , p3_df.columns[2]: "Parent3_UI" , p3_df.columns[3]: "Parent3"})
p3_df.head()

,Source,Child_UI,Parent3_UI,Parent3
0,SNOMEDCT_US,90507008,248727005,Venous finding
1,SNOMEDCT_US,90507008,27550009,Vascular disorder
2,SNOMEDCT_US,128052008,27550009,Vascular disorder
3,SNOMEDCT_US,128052008,118948005,Disorder of abdomen
4,SNOMEDCT_US,90507008,248727005,Venous finding


In [69]:
print(len(p3_df))
p3_df = p3_df.drop_duplicates(subset=['Source','Child_UI','Parent3_UI','Parent3'], keep='first')
print(len(p3_df))

111
88


In [71]:
dedup_p3 = np.stack([p3_df['Source'], p3_df['Child_UI'], p3_df['Parent3_UI'], p3_df['Parent3']])

In [90]:
source_df = pd.merge(source_df, p3_df, left_on=['Source','Parent2_UI'], right_on=['Source','Child_UI'],how='left')
source_df = source_df.drop(['Child_UI'], axis = 1)

source_df.head()

,Source,Disease,UID,Synonyms,Parent1_UI,Parent1,Parent2_UI,Parent2,Parent3_UI,Parent3
0,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,248727005,Venous finding
1,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,27550009,Vascular disorder
2,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,128052008,Vascular disease of abdomen,27550009,Vascular disorder
3,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,128052008,Vascular disease of abdomen,118948005,Disorder of abdomen
4,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,90507008,Disorder of vein,248727005,Venous finding


# Get Parents 4 Information

In [81]:
p4 = itertate_child(tgt,dedup_p3)
p4_df = pd.DataFrame(list(zip(p4[0],p4[1],p4[2],p4[3])))
p4_df=p4_df.rename(columns={ p4_df.columns[0]: "Source" ,p4_df.columns[1]: "Child_UI" , p4_df.columns[2]: "Parent4_UI" , p4_df.columns[3]: "Parent4"})
p4_df.head()

,Source,Child_UI,Parent4_UI,Parent4
0,SNOMEDCT_US,248727005,21829004,Blood vessel finding
1,SNOMEDCT_US,27550009,19660004,Disorder of soft tissue
2,SNOMEDCT_US,27550009,21829004,Blood vessel finding
3,SNOMEDCT_US,27550009,49601007,Disorder of cardiovascular system
4,SNOMEDCT_US,27550009,19660004,Disorder of soft tissue


In [82]:
print(len(p4_df))
p4_df = p4_df.drop_duplicates(subset=['Source','Child_UI','Parent4_UI','Parent4'], keep='first')
print(len(p4_df))

113
79


In [85]:
dedup_p4 = np.stack([p4_df['Source'], p4_df['Child_UI'], p4_df['Parent4_UI'], p4_df['Parent4']])

In [91]:
source_df = pd.merge(source_df, p4_df, left_on=['Source','Parent3_UI'], right_on=['Source','Child_UI'],how='left')
source_df = source_df.drop(['Child_UI'], axis = 1)

source_df.head()

,Source,Disease,UID,Synonyms,Parent1_UI,Parent1,Parent2_UI,Parent2,Parent3_UI,Parent3,Parent4_UI,Parent4
0,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,248727005,Venous finding,21829004,Blood vessel finding
1,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,27550009,Vascular disorder,19660004,Disorder of soft tissue
2,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,27550009,Vascular disorder,21829004,Blood vessel finding
3,SNOMEDCT_US,hypertension,34742003,Portal hypertension,234072000,Venous hypertension,90507008,Disorder of vein,27550009,Vascular disorder,49601007,Disorder of cardiovascular system
4,SNOMEDCT_US,hypertension,34742003,Portal hypertension,128056006,Disorder of portal venous system,128052008,Vascular disease of abdomen,27550009,Vascular disorder,19660004,Disorder of soft tissue


In [95]:
source_df.to_csv('Disease with 4lvl.csv',index=False)


In [96]:
end_time = datetime.now()

In [97]:
print('The total time taken to run this workbook is:',(end_time-start_time))

The total time taken to run this workbook is: 1:02:22.492975
